In [1]:
#Assignment 3 Task 1 By Kevin Wong
#CS4210 Machine Learning and its Applications - Summer Semester 2024

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
# Determine if CUDA (GPU) is available and set the device accordingly

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# Step 1: Data Preprocessing

# Define transforms
transform_train = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL images to tensors
    transforms.RandomHorizontalFlip(),  # Apply random horizontal flip
    transforms.RandomRotation(5),  # Apply random rotation
    transforms.Normalize((0.5,), (0.5,))  # Normalize the image pixel values to have mean=0.5 and std=0.5
])

# Define transformations for validation and test data (only normalization)
transform_val_test = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the data
])

# Custom dataset class to handle image data and apply transforms
class CustomImageDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image) # Apply the specified transformations to the image
        if self.labels is not None:
            label = self.labels[idx]
            return image, label # Return both the image and the corresponding label
        return image # For test data, only return the image


# Load datasets from CSV files
train_data = pd.read_csv('train_data.csv', header=None)
train_target = pd.read_csv('train_target.csv', header=None)
test_data = pd.read_csv('test_data.csv', header=None)

# Reshape the data (each row is 2304 pixels, reshape to 48x48 images)
train_data = train_data.values.reshape(-1, 48, 48).astype('float32')
test_data = test_data.values.reshape(-1, 48, 48).astype('float32')

# Split training data into training and validation sets
train_data, val_data, train_target, val_target = train_test_split(
    train_data, train_target.values.flatten(), test_size=0.2, random_state=42)

# Convert numpy arrays to torch tensors and apply transforms
train_dataset = CustomImageDataset(train_data, train_target, transform=transform_train)
val_dataset = CustomImageDataset(val_data, val_target, transform=transform_val_test)
test_dataset = CustomImageDataset(test_data, transform=transform_val_test)

# Create DataLoader objects for batching and shuffling the data during training and evaluation
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [4]:
# Step 2: Building the Model

# Define a Convolutional Neural Network (CNN) architecture for image classification
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        # Fully connected layers with dropout for regularization
        self.fc1 = nn.Linear(512*3*3, 512)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(512, 3)  # 3 classes: Angry, Happy, Neutral
        
    def forward(self, x):
        # Define the forward pass using Leaky ReLU activation functions and max pooling
        x = F.leaky_relu(F.max_pool2d(self.bn1(self.conv1(x)), 2), negative_slope=0.01)
        x = F.leaky_relu(F.max_pool2d(self.bn2(self.conv2(x)), 2), negative_slope=0.01)
        x = F.leaky_relu(F.max_pool2d(self.bn3(self.conv3(x)), 2), negative_slope=0.01)
        x = F.leaky_relu(F.max_pool2d(self.bn4(self.conv4(x)), 2), negative_slope=0.01)
        x = x.view(-1, 512*3*3) # Flatten the output from the convolutional layers
        x = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x = self.dropout(x)  # Apply dropout for regularization
        x = self.fc2(x) # Final output layer
        return x

In [5]:
# Instantiate the model, define loss function and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss() # Use cross-entropy loss for multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) # Use Adam optimizer with weight decay

# Step 3: Training the Model

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Clear the gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels) # Calculate the loss
            loss.backward() # Backward pass to calculate the gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Clip gradients to avoid exploding gradients
            optimizer.step() # Update the weights
            running_loss += loss.item()
        
        val_loss, val_accuracy = validate_model(model, val_loader, criterion) # Validate the model on the validation set
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')



def validate_model(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for images, labels in val_loader:
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels) # Calculate the loss
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1) # Get the predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # Calculate the number of correct predictions
    
    val_loss /= len(val_loader) # Calculate average validation loss
    val_accuracy = 100 * correct / total # Calculate validation accuracy
    return val_loss, val_accuracy

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch [1/20], Loss: 1.2308, Validation Loss: 0.9735, Validation Accuracy: 53.57%
Epoch [2/20], Loss: 0.8160, Validation Loss: 0.6824, Validation Accuracy: 70.17%
Epoch [3/20], Loss: 0.6942, Validation Loss: 0.6173, Validation Accuracy: 73.48%
Epoch [4/20], Loss: 0.6406, Validation Loss: 0.6376, Validation Accuracy: 72.02%
Epoch [5/20], Loss: 0.6043, Validation Loss: 0.6544, Validation Accuracy: 73.04%
Epoch [6/20], Loss: 0.5716, Validation Loss: 0.5588, Validation Accuracy: 76.88%
Epoch [7/20], Loss: 0.5473, Validation Loss: 0.5569, Validation Accuracy: 76.54%
Epoch [8/20], Loss: 0.5283, Validation Loss: 0.5575, Validation Accuracy: 77.90%
Epoch [9/20], Loss: 0.4996, Validation Loss: 0.5639, Validation Accuracy: 75.89%
Epoch [10/20], Loss: 0.4869, Validation Loss: 0.5692, Validation Accuracy: 77.19%
Epoch [11/20], Loss: 0.4695, Validation Loss: 0.5496, Validation Accuracy: 77.37%
Epoch [12/20], Loss: 0.4588, Validation Loss: 0.5665, Validation Accuracy: 77.50%
Epoch [13/20], Loss: 0.42

In [6]:
# Step 4: Making Predictions

def make_predictions(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    with torch.no_grad(): # Disable gradient calculation
        for images in test_loader:
            outputs = model(images) # Forward pass
            _, predicted = torch.max(outputs.data, 1) # Get the predicted class
            predictions.extend(predicted.cpu().numpy())  # Store the predictions
    return predictions

# Generate predictions on the test set
predictions = make_predictions(model, test_loader)

# Prepare submission file
submission_df = pd.DataFrame({'Id': np.arange(len(predictions)), 'Category': predictions})
submission_df.to_csv('submission.csv', index=False)
print("Submission file created!")

Submission file created!
